In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dev-testing/EXIST2025_dev.json
/kaggle/input/datasets-translated/EXIST2025_training_task1_1_gold_soft.json
/kaggle/input/datasets-translated/EXIST2025_training_translated_en.json
/kaggle/input/datasets-translated/EXIST2025_training_translated_es.json
/kaggle/input/dev-gold-hard/EXIST2025_dev_task1_1_gold_hard.json
/kaggle/input/dev-gold-soft/EXIST2025_dev_task1_1_gold_soft.json


In [2]:
import wandb

wandb.login(key="a40bf999db96c982783dc52dd0594d3347848f02")


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: faisalsara124 (faisalsara124-habib-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Newer version

In [18]:
print(data_en[0])

{'id': '100001', 'tweet': '@TheChiflis Ignora to the other, he\'s a jerk.The problem with this youtuber denounces harassment... when it doesn\'t affect left-wing people.For example, in his video about the gamergate he presents as "normal" the harassment that Fisher, Anita or Zöey receive when there were even bomb threats.'}


In [ ]:
import json
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from torch.utils.data import Dataset

# === Load Tweets ===
with open("/kaggle/working/augment_en.json", "r", encoding="utf-8") as f:
    data_en = json.load(f)

with open("/kaggle/working/augment_es.json", "r", encoding="utf-8") as f:
    data_es = json.load(f)

# === Load gold_soft_train ===
with open("/kaggle/working/augment_gold.json", "r", encoding="utf-8") as f:
    gold_soft = json.load(f)

# Convert gold_soft to a dict for fast access
gold_soft_dict = {entry["id"]: entry["value"] for entry in gold_soft}

# Define binary labels
CORRECT_LABELS = ["YES", "NO"]

# === Process Tweets with Corresponding Soft Labels ===
def process_data_with_soft_labels(data):
    tweets = []
    labels = []
    ids = []

    for entry in data:
        tweet_id = entry["id"]
        tweet = entry["tweet"]

        if tweet_id not in gold_soft_dict:
            continue  # Skip if soft label not found

        soft_label_dict = gold_soft_dict[tweet_id]

        # Build binary soft label vector [YES_score, NO_score]
        soft_label_vector = [soft_label_dict.get(label, 0.0) for label in CORRECT_LABELS]

        tweets.append(tweet)
        labels.append(soft_label_vector)
        ids.append(tweet_id)

    return tweets, labels, ids

# Process both English and Spanish tweets
tweets_en, labels_en, ids_en = process_data_with_soft_labels(data_en)
tweets_es, labels_es, ids_es = process_data_with_soft_labels(data_es)

# === Tokenizer ===
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# === Custom Dataset Class ===
class TweetDataset(Dataset):
    def __init__(self, texts, labels, ids, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.ids = ids
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        tweet_id = self.ids[idx]
        labels = torch.tensor(self.labels[idx], dtype=torch.float)
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')

        return {
            "id": tweet_id,
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": labels
        }

# === Train-validation split ===
def get_datasets(tweets, labels, ids):
    train_texts, val_texts, train_labels, val_labels, train_ids, val_ids = train_test_split(
        tweets, labels, ids, test_size=0.2, random_state=42
    )
    train_dataset = TweetDataset(train_texts, train_labels, train_ids, tokenizer)
    val_dataset = TweetDataset(val_texts, val_labels, val_ids, tokenizer)
    return train_dataset, val_dataset

# === Create datasets ===
train_dataset_en, val_dataset_en = get_datasets(tweets_en, labels_en, ids_en)
train_dataset_es, val_dataset_es = get_datasets(tweets_es, labels_es, ids_es)

# === Train Model ===
def train_model(train_dataset, val_dataset, output_dir):
    model = AutoModelForSequenceClassification.from_pretrained(
        "bert-base-multilingual-cased",
        num_labels=2
        
    )

    training_args = TrainingArguments(
    output_dir=output_dir,
    do_train=True,
    do_eval=True,
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    logging_steps=250,
    save_total_limit=2
)


    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    trainer.train()
    return trainer

# === Train English and Spanish models ===
trainer_en = train_model(train_dataset_en, val_dataset_en, output_dir="./results/en")
trainer_es = train_model(train_dataset_es, val_dataset_es, output_dir="./results/es")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
250,0.581300


# **Dev Testing starts from here**

In [17]:
with open("/kaggle/input/dev-testing/EXIST2025_dev.json", "r", encoding="utf-8") as f:
    dev_data = json.load(f)

# Extract tweets and IDs
dev_tweets = [entry["tweet"] for entry in dev_data.values()]
dev_ids = [entry["id_EXIST"] for entry in dev_data.values()]


In [18]:
import json

# Load the dev dataset
with open("/kaggle/input/dev-testing/EXIST2025_dev.json", "r", encoding="utf-8") as f:
    dev_data = json.load(f)

# Split into English & Spanish
english_dev_tweets = []
english_dev_ids = []
spanish_dev_tweets = []
spanish_dev_ids = []

for entry in dev_data.values():
    tweet_id = entry["id_EXIST"]
    tweet = entry["tweet"]
    lang = entry["lang"]

    if lang == "en":
        english_dev_tweets.append(tweet)
        english_dev_ids.append(tweet_id)
    elif lang == "es":
        spanish_dev_tweets.append(tweet)
        spanish_dev_ids.append(tweet_id)

# Debugging: Check split sizes
print(f"English Dev Samples: {len(english_dev_tweets)}")
print(f"Spanish Dev Samples: {len(spanish_dev_tweets)}")


English Dev Samples: 489
Spanish Dev Samples: 549


In [19]:
import os
from transformers import BertForSequenceClassification

# Function to get the latest checkpoint
def get_latest_checkpoint(directory="./results"):
    checkpoints = [d for d in os.listdir(directory) if d.startswith("checkpoint-")]
    if not checkpoints:
        raise ValueError(f"No checkpoints found in {directory}")
    latest_checkpoint = sorted(checkpoints, key=lambda x: int(x.split('-')[-1]))[-1]
    return os.path.join(directory, latest_checkpoint)

# Load the best model checkpoint for English and Spanish
latest_checkpoint_en = get_latest_checkpoint("./results/en")
latest_checkpoint_es = get_latest_checkpoint("./results/es")

print(f"Using latest checkpoint for English: {latest_checkpoint_en}")
print(f"Using latest checkpoint for Spanish: {latest_checkpoint_es}")

# Load models
model_en = BertForSequenceClassification.from_pretrained(latest_checkpoint_en)
model_es = BertForSequenceClassification.from_pretrained(latest_checkpoint_es)


Using latest checkpoint for English: ./results/en/checkpoint-519
Using latest checkpoint for Spanish: ./results/es/checkpoint-519


# This is Soft Soft

In [20]:

def predict_on_dev(tweets, ids, model, tokenizer, label_classes, output_file):
    model.eval()
    results = []

    for tweet, tweet_id in zip(tweets, ids):
        encoding = tokenizer(tweet, truncation=True, padding="max_length", max_length=256, return_tensors="pt")

        with torch.no_grad():
            outputs = model(**encoding)

        logits = outputs.logits.squeeze()
        probs = torch.sigmoid(logits).cpu().numpy()

        # Convert probabilities to dictionary format and sort by highest probability
        soft_label_dict = {label_classes[i]: float(probs[i]) for i in range(len(label_classes))}
        sorted_soft_label_dict = dict(sorted(soft_label_dict.items(), key=lambda item: item[1], reverse=True))  # Sort descending

        results.append({
            "test_case": "EXIST2025",
            "id": tweet_id,
            "value": sorted_soft_label_dict  # Rename "soft_label" to "value" and sort it
        })

    # Save results
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=4)

    print(f"Predictions saved to {output_file}")
    
label_classes = CORRECT_LABELS


# Run predictions
predict_on_dev(english_dev_tweets, english_dev_ids, model_en, tokenizer, label_classes, "EXIST2025_dev_predictions_en.json")
predict_on_dev(spanish_dev_tweets, spanish_dev_ids, model_es, tokenizer, label_classes, "EXIST2025_dev_predictions_es.json")


Predictions saved to EXIST2025_dev_predictions_en.json
Predictions saved to EXIST2025_dev_predictions_es.json


# This be Hard Hard

In [8]:
def predict_hard_labels_from_soft_model(tweets, ids, model, tokenizer, label_classes, output_file):
    """
    Uses the soft model to predict a single hard label: "YES" or "NO".
    - Assigns the label with the higher probability.
    """
    model.eval()
    results = []

    for tweet, tweet_id in zip(tweets, ids):
        encoding = tokenizer(tweet, truncation=True, padding="max_length", max_length=256, return_tensors="pt")

        with torch.no_grad():
            outputs = model(**encoding)

        logits = outputs.logits.squeeze()
        probs = torch.sigmoid(logits).cpu().numpy()

        # Pick the label with the highest probability (YES or NO)
        max_index = int(probs.argmax())
        predicted_label = label_classes[max_index]

        results.append({
            "test_case": "EXIST2025",
            "id": tweet_id,
            "value": [predicted_label]  # Only one label
        })

    # Save results
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=4)

    print(f"Hard label predictions saved to {output_file}")

predict_hard_labels_from_soft_model(english_dev_tweets, english_dev_ids, model_en, tokenizer, label_classes, "EXIST2025_dev_predictions_hard_en.json")
predict_hard_labels_from_soft_model(spanish_dev_tweets, spanish_dev_ids, model_es, tokenizer, label_classes, "EXIST2025_dev_predictions_hard_es.json")


Hard label predictions saved to EXIST2025_dev_predictions_hard_en.json
Hard label predictions saved to EXIST2025_dev_predictions_hard_es.json


# Merging soft models ka dev set predictions

In [21]:
import json

# Load the Spanish predictions
with open("/kaggle/working/EXIST2025_dev_predictions_es.json", "r", encoding="utf-8") as f:
    es_data = json.load(f)

# Load the English predictions
with open("/kaggle/working/EXIST2025_dev_predictions_en.json", "r", encoding="utf-8") as f:
    en_data = json.load(f)

# Assuming both files contain lists of predictions, merge them
if isinstance(es_data, list) and isinstance(en_data, list):
    merged_data = es_data + en_data
else:
    raise ValueError("JSON structure is not a list. Ensure both files contain lists.")

# Save to a new file
output_filename = "EXIST2025_dev_predictions_merged_soft.json"
with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(merged_data, f, indent=4, ensure_ascii=False)

print(f"Merging complete! Saved to {output_filename}")


Merging complete! Saved to EXIST2025_dev_predictions_merged_soft.json


In [22]:
import json
import numpy as np

# Load your predictions file
with open('EXIST2025_dev_predictions_merged_soft.json', 'r') as f:
    predictions = json.load(f)

# Define the snapping values (multiples of 1/6)
snap_vals = np.array([i / 6 for i in range(7)])  # [0.0, 0.1667, ..., 1.0]

def snap_to_nearest_sixth(value):
    return float(snap_vals[np.argmin(np.abs(snap_vals - value))])

# Snap each value in the 'value' dict
for entry in predictions:
    entry['value'] = {k: snap_to_nearest_sixth(v) for k, v in entry['value'].items()}

# Save the snapped predictions to a new file
with open('EXIST2025_dev_predictions_snapped_soft.json', 'w') as f:
    json.dump(predictions, f, indent=2)


# Merging hard models ka dev set predictions

In [11]:
import json

# Load the Spanish predictions
with open("/kaggle/working/EXIST2025_dev_predictions_hard_es.json", "r", encoding="utf-8") as f:
    es_data = json.load(f)

# Load the English predictions
with open("/kaggle/working/EXIST2025_dev_predictions_hard_en.json", "r", encoding="utf-8") as f:
    en_data = json.load(f)

# Assuming both files contain lists of predictions, merge them
if isinstance(es_data, list) and isinstance(en_data, list):
    merged_data = es_data + en_data
else:
    raise ValueError("JSON structure is not a list. Ensure both files contain lists.")

# Save to a new file
output_filename = "EXIST2025_dev_predictions_merged_hard.json"
with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(merged_data, f, indent=4, ensure_ascii=False)

print(f"Merging complete! Saved to {output_filename}")


Merging complete! Saved to EXIST2025_dev_predictions_merged_hard.json


In [12]:
import json

def convert_prediction_format(input_file, output_file):
    with open(input_file, "r", encoding="utf-8") as f:
        predictions = json.load(f)

    converted = []
    for entry in predictions:
        # Convert the "value" list to a single string (first label only)
        new_entry = {
            "test_case": entry["test_case"],
            "id": entry["id"],
            "value": entry["value"][0] if isinstance(entry["value"], list) else entry["value"]
        }
        converted.append(new_entry)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(converted, f, indent=4)

    print(f"Predictions converted to gold format and saved to {output_file}")

convert_prediction_format("EXIST2025_dev_predictions_merged_hard.json", "EXIST2025_dev_predictions_merged_hard_flat.json")


Predictions converted to gold format and saved to EXIST2025_dev_predictions_merged_hard_flat.json


# AEDA Technique

In [10]:
pip install nltk transformers torch tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:000:00:01:03mm
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

# Rija AEDA 

In [14]:
import json
import torch
import random
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from torch.utils.data import Dataset
from collections import defaultdict

# === AEDA helper ===
PUNCTUATIONS = ['.', ',', '!', '?', ';', ':']
def aeda(sentence, num_insertions=3):
    words = sentence.split()
    if not words:
        return sentence
    new_words = words.copy()
    for _ in range(num_insertions):
        insert_pos = random.randint(0, len(new_words))
        punct = random.choice(PUNCTUATIONS)
        new_words.insert(insert_pos, punct)
    return ' '.join(new_words)

# === Load Data ===
with open("/kaggle/input/datasets-translated/EXIST2025_training_translated_en.json", "r", encoding="utf-8") as f:
    data_en = json.load(f)
with open("/kaggle/input/datasets-translated/EXIST2025_training_translated_es.json", "r", encoding="utf-8") as f:
    data_es = json.load(f)
with open("/kaggle/input/datasets-translated/EXIST2025_training_task1_1_gold_soft.json", "r", encoding="utf-8") as f:
    gold_soft = json.load(f)

gold_soft_dict = {entry["id"]: entry["value"] for entry in gold_soft}
label_classes = [
    "IDEOLOGICAL-INEQUALITY",
    "MISOGYNY-NON-SEXUAL-VIOLENCE",
    "OBJECTIFICATION",
    "SEXUAL-VIOLENCE",
    "STEREOTYPING-DOMINANCE",
    "NO"  # Represents non-sexist tweets (previously "-")
]

# === Count Label Distribution ===
label_counts = defaultdict(int)
for soft in gold_soft_dict.values():
    max_label = max(soft, key=soft.get)
    label_counts[max_label] += 1

# === Identify underrepresented labels (you can tune this threshold) ===
avg_count = np.mean(list(label_counts.values()))
underrepresented_labels = [label for label, count in label_counts.items() if count < avg_count]

# === Process Tweets & Augment Underrepresented Only ===
def process_data_with_soft_labels(data, augment=True, augment_n=2):
    tweets, labels, ids = [], [], []

    for entry in data.values():
        tweet_id = entry["id_EXIST"]
        tweet = entry["tweet"]

        if tweet_id not in gold_soft_dict:
            continue

        soft_label_dict = gold_soft_dict[tweet_id]
        soft_label_vector = [soft_label_dict.get(label, 0.0) for label in label_classes]

        # Original tweet
        tweets.append(tweet)
        labels.append(soft_label_vector)
        ids.append(tweet_id)

        # Determine primary label
        main_label = max(soft_label_dict, key=soft_label_dict.get)

        # Augment only if underrepresented
        if augment and main_label in underrepresented_labels:
            for i in range(augment_n):
                augmented_tweet = aeda(tweet)
                tweets.append(augmented_tweet)
                labels.append(soft_label_vector)
                ids.append(f"{tweet_id}_aug{i+1}")

    return tweets, labels, ids

# Process English and Spanish data
tweets_en, labels_en, ids_en = process_data_with_soft_labels(data_en)
tweets_es, labels_es, ids_es = process_data_with_soft_labels(data_es)

class TweetDataset(Dataset):
    def __init__(self, texts, labels, ids, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.ids = ids
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        tweet_id = self.ids[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float)
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            "id": tweet_id,
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": label
        }

# === Train-validation split ===
def get_datasets(tweets, labels, ids):
    train_texts, val_texts, train_labels, val_labels, train_ids, val_ids = train_test_split(
        tweets, labels, ids, test_size=0.2, random_state=42
    )
    train_dataset = TweetDataset(train_texts, train_labels, train_ids, tokenizer)
    val_dataset = TweetDataset(val_texts, val_labels, val_ids, tokenizer)
    return train_dataset, val_dataset

# === Create datasets ===
train_dataset_en, val_dataset_en = get_datasets(tweets_en, labels_en, ids_en)
train_dataset_es, val_dataset_es = get_datasets(tweets_es, labels_es, ids_es)

print(f"English train set size: {len(train_dataset_en)} (with selective augmentation)")
print(f"Spanish train set size: {len(train_dataset_es)} (with selective augmentation)")

English train set size: 10496 (with selective augmentation)
Spanish train set size: 10496 (with selective augmentation)


In [43]:
# from transformers import BertForSequenceClassification

# model = BertForSequenceClassification.from_pretrained("./results/en")  # or ./results/es
import os

def get_latest_checkpoint(path):
    checkpoints = [d for d in os.listdir(path) if d.startswith("checkpoint-")]
    checkpoints = sorted(checkpoints, key=lambda x: int(x.split("-")[1]))
    return os.path.join(path, checkpoints[-1]) if checkpoints else None

latest_ckpt = get_latest_checkpoint("./results/en")
model = BertForSequenceClassification.from_pretrained(latest_ckpt)



In [44]:
def fine_tune_model(model, train_dataset, val_dataset, output_dir):
    training_args = TrainingArguments(
        output_dir=output_dir,
        do_train=True,
        do_eval=True,
        num_train_epochs=2,  # You can reduce this, since it's fine-tuning
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        logging_dir="./logs",
        logging_steps=150,
        save_total_limit=2
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    trainer.train()
    return trainer


In [29]:
print(train_dataset_en[0])

{'id': '200294_aug2', 'text': '@mtgreenee Why, you have personally demonstrated your capabilities and desire , to use advanced ; ! weapons. Are you saying your not as good as a man when it comes to warfare? Pick a lane.', 'labels': tensor([0.8333, 0.1667])}


In [ ]:
latest_ckpt = get_latest_checkpoint("./results/en")
# Load from base training checkpoint
# model_en = BertForSequenceClassification.from_pretrained(latest_ckpt)
model_en = BertForSequenceClassification.from_pretrained(
    latest_ckpt,
    num_labels=2,
    problem_type="multi_label_classification"
)
latest_ckpt = get_latest_checkpoint("./results/es")
# model_es = BertForSequenceClassification.from_pretrained(latest_ckpt, num_labels=2)
model_es = BertForSequenceClassification.from_pretrained(
    latest_ckpt,
    num_labels=2,
    problem_type="multi_label_classification"
)

# Fine-tune on augmented data
trainer_en_aug = fine_tune_model(model_en, train_dataset_en, val_dataset_en, output_dir="./results/en_finetuned")
trainer_es_aug = fine_tune_model(model_es, train_dataset_es, val_dataset_es, output_dir="./results/es_finetuned")


In [29]:
pip install pyevall

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.6/894.6 kB 18.7 MB/s eta 0:00:0000:01
  Created wheel for pyevall: filename=PyEvALL-0.1.78-py3-none-any.whl size=34777 sha256=2d636256d467108c6d85ca895b690e7c5fc8913dbb25a87df2f69e73eb19226e
  Stored in directory: /root/.cache/pip/wheels/1f/71/ae/bf04901d4d2616e45aee909fba432a0ef67dd0d276f86eaab7
  Created wheel for jsbeautifier: filename=jsbeautifier-1.14.9-py3-none-any.whl size=94155 sha256=bfc93093cdcaca2c3427c0cc37cb063475f2670b03f825a9d01a5d2baf7f27b3
  Stored in directory: /root/.cache/pip/wheels/b9/bd/4d/f11d410af889a445a4f36209ecc5fa5802aa9040a0bef2b235
Successfully built pyevall jsbeautifier
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
  

In [39]:
# Step 1: Use your existing trainer to generate predictions
predictions_output = model_en.predict(val_dataset_en)

# Step 2: Convert logits to probabilities using softmax
import torch
import torch.nn.functional as F

soft_preds = F.softmax(torch.tensor(predictions_output.predictions), dim=1).tolist()

# Step 3: Get tweet IDs
# tweet_ids = val_dataset_en.ids  # This must match how you constructed val_dataset_en
# Ensure IDs and predictions are unique and aligned
seen_ids = set()
filtered_preds = []
filtered_ids = []

for i, tweet_id in enumerate(val_dataset_en.ids):
    if tweet_id not in seen_ids:
        seen_ids.add(tweet_id)
        filtered_ids.append(tweet_id)
        filtered_preds.append(soft_preds[i])


# Step 4: Format predictions for PyEvALL
predictions_json = []
for tweet_id, probs in zip(filtered_ids, filtered_preds):
    predictions_json.append({
        "id": tweet_id,
        "test_case": "EXIST2025",  # Use exact match from gold
        "value": {
            "sexist": probs[0],
            "not-sexist": probs[1]
        }
    })

# Step 5: Save predictions to JSON
output_path = "/kaggle/working/EXIST2025_dev_predictions_merged_soft_aeda.json"
with open(output_path, "w") as f:
    json.dump(predictions_json, f, indent=2)


In [38]:
import json
with open("/kaggle/input/dev-gold-soft/EXIST2025_dev_task1_1_gold_soft.json") as f:
    print(json.load(f)[:5])


[{'test_case': 'EXIST2025', 'id': '300001', 'value': {'NO': 0.5, 'YES': 0.5}}, {'test_case': 'EXIST2025', 'id': '300002', 'value': {'YES': 0.8333333333333334, 'NO': 0.16666666666666666}}, {'test_case': 'EXIST2025', 'id': '300003', 'value': {'NO': 1.0, 'YES': 0.0}}, {'test_case': 'EXIST2025', 'id': '300004', 'value': {'NO': 0.16666666666666666, 'YES': 0.8333333333333334}}, {'test_case': 'EXIST2025', 'id': '300005', 'value': {'NO': 0.8333333333333334, 'YES': 0.16666666666666666}}]


In [40]:
from pyevall.evaluation import PyEvALLEvaluation
from pyevall.utils.utils import PyEvALLUtils

# Use the new AEDA-based model predictions
predictions = "/kaggle/working/EXIST2025_dev_predictions_merged_soft_aeda.json"
gold = "/kaggle/input/dev-gold-soft/EXIST2025_dev_task1_1_gold_soft.json"  # No change

evaluator = PyEvALLEvaluation()

params = dict()
params[PyEvALLUtils.PARAM_REPORT] = PyEvALLUtils.PARAM_OPTION_REPORT_EMBEDDED

metrics = ["ICMSoft", "ICMSoftNorm", "CrossEntropy"]

# Run evaluation
report = evaluator.evaluate(predictions, gold, metrics, **params)
report.print_report()


2025-05-08 18:07:05,575 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICMSoft', 'ICMSoftNorm', 'CrossEntropy']
2025-05-08 18:07:06,396 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM Soft evaluation method
2025-05-08 18:07:07,637 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM-Soft Normalized evaluation method
2025-05-08 18:07:07,640 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM Soft evaluation method
2025-05-08 18:07:09,348 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM Soft evaluation method
2025-05-08 18:07:10,193 - pyevall.metrics.metrics - INFO -             evaluate() - Executing Cross Entropy evaluation method
{
  "metrics": {
    "ICMSoft": {
      "name": "Information Contrast Model Soft",
      "acronym": "ICM-Soft",
      "description": "Coming soon!",
      "status": "OK",
      "results": {
        "test_cases": [{
         

# Metric Calculation

In [23]:
pip install pyevall

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.6/894.6 kB 24.5 MB/s eta 0:00:0000:01
  Created wheel for pyevall: filename=PyEvALL-0.1.78-py3-none-any.whl size=34777 sha256=f57ae11799d5f59797d404e0f48dc8cd93c2c958377b6108cbf8c083e567df4b
  Stored in directory: /root/.cache/pip/wheels/1f/71/ae/bf04901d4d2616e45aee909fba432a0ef67dd0d276f86eaab7
  Created wheel for jsbeautifier: filename=jsbeautifier-1.14.9-py3-none-any.whl size=94155 sha256=ae25faf1d5207013d9c3ad8dbdd3444197f65dc10fa23300e97221263e98edf1
  Stored in directory: /root/.cache/pip/wheels/b9/bd/4d/f11d410af889a445a4f36209ecc5fa5802aa9040a0bef2b235
Successfully built pyevall jsbeautifier
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
  

# This is hard metrics

In [24]:
from pyevall.evaluation import PyEvALLEvaluation
from pyevall.utils.utils import PyEvALLUtils

# Define file paths
predictions = "/kaggle/working/EXIST2025_dev_predictions_merged_hard_flat.json"
gold = "/kaggle/input/dev-gold-hard/EXIST2025_dev_task1_1_gold_hard.json"

# Initialize evaluator
evaluator = PyEvALLEvaluation()

# Set evaluation parameters
params = dict()
params[PyEvALLUtils.PARAM_REPORT] = PyEvALLUtils.PARAM_OPTION_REPORT_EMBEDDED  # Embedded report

# Define metrics for hard-label binary classification
metrics = ["ICM", "ICMNorm", "FMeasure"]

# Run evaluation
report = evaluator.evaluate(predictions, gold, metrics, **params)

# Print report
report.print_report()


2025-05-08 09:03:59,566 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM', 'ICMNorm', 'FMeasure']


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/EXIST2025_dev_predictions_merged_hard_flat.json'

# These are soft metrics

In [25]:
from pyevall.evaluation import PyEvALLEvaluation
from pyevall.utils.utils import PyEvALLUtils

# Define file paths
predictions = "/kaggle/working/EXIST2025_dev_predictions_merged_soft.json"  # Your prediction file
gold = "/kaggle/input/dev-gold-soft/EXIST2025_dev_task1_1_gold_soft.json"    # Your gold file

# Initialize PyEvALL evaluation
evaluator = PyEvALLEvaluation()

# Set evaluation parameters
params = dict()
params[PyEvALLUtils.PARAM_REPORT] = PyEvALLUtils.PARAM_OPTION_REPORT_EMBEDDED  # Embedded report

# Define evaluation metrics for soft binary classification
metrics = ["ICMSoft", "ICMSoftNorm", "CrossEntropy"]

# Run evaluation
report = evaluator.evaluate(predictions, gold, metrics, **params)

# Print evaluation report
report.print_report()


2025-05-08 09:04:08,178 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICMSoft', 'ICMSoftNorm', 'CrossEntropy']
2025-05-08 09:04:08,430 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM Soft evaluation method
2025-05-08 09:04:09,339 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM-Soft Normalized evaluation method
2025-05-08 09:04:09,343 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM Soft evaluation method
2025-05-08 09:04:10,243 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM Soft evaluation method
2025-05-08 09:04:11,115 - pyevall.metrics.metrics - INFO -             evaluate() - Executing Cross Entropy evaluation method
cargado 29
{
  "metrics": {
    "ICMSoft": {
      "name": "Information Contrast Model Soft",
      "acronym": "ICM-Soft",
      "description": "Coming soon!",
      "status": "OK",
      "results": {
        "test_cases": [

# Finetuning with ensemble model

In [21]:
import json
import torch
import numpy as np
import os
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from typing import List, Dict, Tuple

# === Configuration ===
CORRECT_LABELS = ["YES", "NO"]
MAX_LENGTH = 256
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_latest_checkpoint(model_dir: str) -> str:
    """Find the latest checkpoint directory in a model directory."""
    checkpoints = [d for d in os.listdir(model_dir) if d.startswith("checkpoint-")]
    if not checkpoints:
        raise ValueError(f"No checkpoints found in {model_dir}")
    # Sort by checkpoint number (extract the number after 'checkpoint-')
    checkpoints.sort(key=lambda x: int(x.split("-")[1]))
    return os.path.join(model_dir, checkpoints[-1])

# === Load Data ===
def load_data(translated_path: str, gold_soft_path: str) -> Tuple[List[str], List[List[float]], List[str]]:
    """Load and process data with soft labels."""
    with open(translated_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    with open(gold_soft_path, "r", encoding="utf-8") as f:
        gold_soft = json.load(f)
    
    gold_soft_dict = {entry["id"]: entry["value"] for entry in gold_soft}
    
    tweets, labels, ids = [], [], []
    for entry in data.values():
        tweet_id = entry["id_EXIST"]
        if tweet_id in gold_soft_dict:
            soft_label_dict = gold_soft_dict[tweet_id]
            labels.append([soft_label_dict.get(label, 0.0) for label in CORRECT_LABELS])
            tweets.append(entry["tweet"])
            ids.append(tweet_id)
    
    return tweets, labels, ids

# === Ensemble Model Class ===
class EnsembleSoftClassifier:
    def __init__(self, model_paths: List[str], tokenizer_path: str = "bert-base-multilingual-cased"):
        self.models = []
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
        
        for path in model_paths:
            # Handle local paths correctly
            if os.path.isdir(path):
                # Find the latest checkpoint
                checkpoint_path = get_latest_checkpoint(path)
                # Verify model files exist in the checkpoint
                if not os.path.exists(os.path.join(checkpoint_path, "pytorch_model.bin")) and \
                   not os.path.exists(os.path.join(checkpoint_path, "model.safetensors")):
                    raise ValueError(f"Model files not found in {checkpoint_path}")
                model = BertForSequenceClassification.from_pretrained(checkpoint_path)
            else:
                # For HF model IDs
                model = BertForSequenceClassification.from_pretrained(path)
                
            model.to(DEVICE)
            model.eval()
            self.models.append(model)
    
    def predict_proba(self, texts: List[str]) -> np.ndarray:
        """Predict soft probabilities using ensemble averaging."""
        encodings = self.tokenizer(texts, truncation=True, padding='max_length', 
                                 max_length=MAX_LENGTH, return_tensors='pt')
        
        input_ids = encodings['input_ids'].to(DEVICE)
        attention_mask = encodings['attention_mask'].to(DEVICE)
        
        all_probs = []
        with torch.no_grad():
            for model in self.models:
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits.detach().cpu()
                probs = torch.sigmoid(logits).numpy()
                all_probs.append(probs)
        
        # Average probabilities across models
        avg_probs = np.mean(all_probs, axis=0)
        return avg_probs

# === Main Execution ===
if __name__ == "__main__":
    # Use paths to the model directories (not the checkpoints)
    model_paths = [
        "/kaggle/working/results/en",  # Your English model directory
        "/kaggle/working/results/es",  # Your Spanish model directory
        "bert-base-multilingual-cased"  # Base model for diversity
    ]
    
    # Step 2: Create ensemble
    ensemble = EnsembleSoftClassifier(model_paths)
    
    # Step 3: Predict on dev set
    predict_ensemble_dev(
        ensemble,
        "/kaggle/input/dev-testing/EXIST2025_dev.json",
        "EXIST2025_dev_predictions_ensemble_soft.json"
    )
    
    # Optional: Snap probabilities to nearest 1/6
    with open("EXIST2025_dev_predictions_ensemble_soft.json", "r") as f:
        predictions = json.load(f)
    
    snap_vals = np.array([i / 6 for i in range(7)])
    for entry in predictions:
        entry['value'] = {k: float(snap_vals[np.argmin(np.abs(snap_vals - v))]) 
                         for k, v in entry['value'].items()}
    
    with open("EXIST2025_dev_predictions_ensemble_soft_snapped.json", "w") as f:
        json.dump(predictions, f, indent=4)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Ensemble predictions saved to EXIST2025_dev_predictions_ensemble_soft.json


# ensemble soft eval

In [26]:
from pyevall.evaluation import PyEvALLEvaluation
from pyevall.utils.utils import PyEvALLUtils

# Define file paths - using your ensemble predictions
predictions = "/kaggle/working/EXIST2025_dev_predictions_ensemble_soft.json"  # Ensemble predictions
gold = "/kaggle/input/dev-gold-soft/EXIST2025_dev_task1_1_gold_soft.json"    # Gold file

# Initialize PyEvALL evaluation
evaluator = PyEvALLEvaluation()

# Set evaluation parameters
params = dict()
params[PyEvALLUtils.PARAM_REPORT] = PyEvALLUtils.PARAM_OPTION_REPORT_EMBEDDED  # Embedded report

# Define evaluation metrics for soft binary classification
metrics = ["ICMSoft", "ICMSoftNorm"]  # Soft metrics for probability outputs

# Run evaluation
print("Evaluating ensemble model performance...")
report = evaluator.evaluate(predictions, gold, metrics, **params)

# Print evaluation report
print("\n=== Ensemble Model Evaluation Results ===")
report.print_report()

# Optional: Also evaluate the snapped version
print("\nEvaluating snapped probabilities version...")
snapped_predictions = "/kaggle/working/EXIST2025_dev_predictions_ensemble_soft_snapped.json"
snapped_report = evaluator.evaluate(snapped_predictions, gold, metrics, **params)
print("\n=== Snapped Probabilities Evaluation Results ===")
snapped_report.print_report()

Evaluating ensemble model performance...
2025-05-08 09:04:29,194 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICMSoft', 'ICMSoftNorm']


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/EXIST2025_dev_predictions_ensemble_soft.json'

# trying to improve ensemble

In [27]:
import json
import torch
import numpy as np
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from typing import List, Dict

# === Configuration ===
CORRECT_LABELS = ["YES", "NO"]
MAX_LENGTH = 256
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16

MODEL_WEIGHTS = {
    "./results/en": 0.6,
    "./results/es": 0.2,
    "roberta-base": 0.2
}

# Base models for tokenizer loading (update if needed)
BASE_MODEL_MAP = {
    "./results/en": "bert-base-uncased",
    "./results/es": "bert-base-uncased",
}

def get_latest_checkpoint(model_dir: str) -> str:
    checkpoints = [d for d in os.listdir(model_dir) if d.startswith("checkpoint-")]
    if not checkpoints:
        raise ValueError(f"No checkpoints found in {model_dir}")
    checkpoints.sort(key=lambda x: int(x.split("-")[1]))
    return os.path.join(model_dir, checkpoints[-1])

class WeightedEnsembleClassifier:
    def __init__(self, model_weights: Dict[str, float]):
        self.model_weights = model_weights
        self.tokenizers = {}

        # Preload tokenizers only
        for model_path in model_weights:
            if os.path.isdir(model_path):
                base_model = BASE_MODEL_MAP.get(model_path)
                if base_model is None:
                    raise ValueError(f"Tokenizer base not found for {model_path}")
                self.tokenizers[model_path] = AutoTokenizer.from_pretrained(base_model)
            else:
                self.tokenizers[model_path] = AutoTokenizer.from_pretrained(model_path)

        # Normalize weights
        total = sum(model_weights.values())
        self.normalized_weights = {k: v / total for k, v in model_weights.items()}

    def predict_proba(self, texts: List[str]) -> np.ndarray:
        all_probs = np.zeros((len(texts), 2))

        for model_path, weight in self.normalized_weights.items():
            tokenizer = self.tokenizers[model_path]
            if os.path.isdir(model_path):
                model_dir = get_latest_checkpoint(model_path)
            else:
                model_dir = model_path

            model = AutoModelForSequenceClassification.from_pretrained(model_dir).to(DEVICE)
            model.eval()

            model_probs = []

            for i in range(0, len(texts), BATCH_SIZE):
                batch = texts[i:i + BATCH_SIZE]
                encodings = tokenizer(batch, padding="max_length", truncation=True,
                                      max_length=MAX_LENGTH, return_tensors="pt").to(DEVICE)

                with torch.no_grad():
                    logits = model(**encodings).logits
                    probs = torch.sigmoid(logits).cpu().numpy()
                    model_probs.extend(probs)

                torch.cuda.empty_cache()

            model_probs = np.array(model_probs)
            all_probs += weight * model_probs

            # Free memory
            del model
            torch.cuda.empty_cache()

        return all_probs

def predict_to_file(ensemble, dev_path: str, output_prefix: str):
    with open(dev_path, "r", encoding="utf-8") as f:
        dev_data = json.load(f)

    all_tweets, all_ids = [], []
    for entry in dev_data.values():
        all_tweets.append(entry["tweet"])
        all_ids.append(entry["id_EXIST"])

    probs = ensemble.predict_proba(all_tweets)

    raw_results = []
    for tweet_id, prob in zip(all_ids, probs):
        raw_results.append({
            "test_case": "EXIST2025",
            "id": tweet_id,
            "value": {CORRECT_LABELS[i]: float(prob[i]) for i in range(len(CORRECT_LABELS))}
        })

    raw_output = f"{output_prefix}_raw.json"
    with open(raw_output, "w", encoding="utf-8") as f:
        json.dump(raw_results, f, indent=4)

    # Snap predictions to nearest i/6
    snap_vals = np.array([i / 6 for i in range(7)])
    snapped_results = []
    for entry in raw_results:
        snapped_results.append({
            "test_case": entry["test_case"],
            "id": entry["id"],
            "value": {k: float(snap_vals[np.argmin(np.abs(snap_vals - v))])
                      for k, v in entry["value"].items()}
        })

    snapped_output = f"{output_prefix}_snapped.json"
    with open(snapped_output, "w", encoding="utf-8") as f:
        json.dump(snapped_results, f, indent=4)

    print(f"✅ Predictions saved to:\n- {raw_output}\n- {snapped_output}")

# === MAIN ===
if __name__ == "__main__":
    ensemble = WeightedEnsembleClassifier(MODEL_WEIGHTS)
    predict_to_file(
        ensemble,
        "/kaggle/input/dev-testing/EXIST2025_dev.json",
        "EXIST2025_dev_predictions_weighted_ensemble"
    )


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Predictions saved to:
- EXIST2025_dev_predictions_weighted_ensemble_raw.json
- EXIST2025_dev_predictions_weighted_ensemble_snapped.json


In [28]:
import json
from pyevall.evaluation import PyEvALLEvaluation
from pyevall.utils.utils import PyEvALLUtils

def evaluate_predictions(prediction_file: str, gold_file: str):
    """Evaluate predictions against gold standard."""
    evaluator = PyEvALLEvaluation()
    params = {PyEvALLUtils.PARAM_REPORT: PyEvALLUtils.PARAM_OPTION_REPORT_EMBEDDED}
    
    print(f"\nEvaluating {prediction_file}...")
    report = evaluator.evaluate(
        prediction_file,
        gold_file,
        ["ICMSoft", "ICMSoftNorm"],
        **params
    )
    report.print_report()

if __name__ == "__main__":
    # File paths - update these based on your actual files
    GOLD_FILE = "/kaggle/input/dev-gold-soft/EXIST2025_dev_task1_1_gold_soft.json"
    
    # Evaluate raw predictions
    evaluate_predictions(
        "EXIST2025_dev_predictions_weighted_ensemble_raw.json",
        GOLD_FILE
    )
    
    # Evaluate snapped predictions
    evaluate_predictions(
        "EXIST2025_dev_predictions_weighted_ensemble_snapped.json",
        GOLD_FILE
    )


Evaluating EXIST2025_dev_predictions_weighted_ensemble_raw.json...
2025-04-24 03:46:23,677 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICMSoft', 'ICMSoftNorm']
2025-04-24 03:46:23,900 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM Soft evaluation method
2025-04-24 03:46:25,241 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM-Soft Normalized evaluation method
2025-04-24 03:46:25,244 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM Soft evaluation method
2025-04-24 03:46:26,074 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM Soft evaluation method
{
  "metrics": {
    "ICMSoft": {
      "name": "Information Contrast Model Soft",
      "acronym": "ICM-Soft",
      "description": "Coming soon!",
      "status": "OK",
      "results": {
        "test_cases": [{
          "name": "EXIST2025",
          "average": -1.4009106699084555
        }],